In [ ]:
from a1chemy.data_source import SinaFinanceClient, EastMoneyClient, XueQiuDataParser
from a1chemy.common import OptionConfig, OptionMap, Underlying
from a1chemy.option.pricing import black_scholes, black_76, gbs
import datetime
import pandas as pd
from IPython.display import display
import ipywidgets as widgets
import numpy as np
import plotly.graph_objects as go


def get_options(symbol, underlying_price, expiry_date_list, filter_strike):
    east_money_client = EastMoneyClient()
    underlying = Underlying(price=underlying_price)
    config = OptionConfig(interest_rate=0.029120, pricing_model=gbs)
    option_map = OptionMap()
    for time_to_expiry in expiry_date_list:
        east_money_client.get_option_data(underlying, config, option_map, symbol, time_to_expiry, filter_strike)
    return option_map

def calculate_positions(underlying, option_map, positions):
    delta = 0
    delta_without_underlying = 0
    gamma = 0
    theta = 0
    vega = 0
    profit = 0
    profit_without_underlying = 0

    for p in positions:
        if p[0] == 'underlying':
            delta += p[1]
            profit += p[1]*(underlying.price-p[2])
        else:
            o = option_map.get_option(p[2], p[3], p[1])
            delta += p[4] * o.delta
            delta_without_underlying += p[4] * o.delta
            gamma += p[4] * o.gamma
            theta += p[4] * o.theta
            vega += p[4] * o.vega
            profit += p[4]*(o.price-p[5])
            profit_without_underlying += p[4]*(o.price-p[5])
    return delta, delta_without_underlying, gamma, theta, vega, profit, profit_without_underlying


def risk_matrix(underlying_price, iv_diff_list, underlying_diff_list, option_map, position):
    risk_matrix_result = []
    for underlying_diff in underlying_diff_list:
        iv_profit_list = ['{:0.3f}({:0.3f})'.format(underlying_price*(1 + underlying_diff), underlying_diff)]
        for iv_diff in iv_diff_list:
            profit = 0
            for p in positions:
                if p[0] is not 'underlying':
                    o = option_map.get_option(p[2], p[3], p[1])
                    new_price = o.recalculate_new_price(iv_diff=o.iv*iv_diff, underlying_price_diff=o.underlying.price*underlying_diff, time_diff=-1)
                    profit += (new_price - o.price) * p[4]
            iv_profit_list.append(int(profit))
        risk_matrix_result.append(iv_profit_list)
    return risk_matrix_result

color_list = [
    '#FF0000',
    '#FF3333',
    '#FF6666',
    '#FF9999',
    '#FFCCCC',
    '#CCFFCC',
    '#99FF99',
    '#66FF66',
    '#33FF33',
    '#00CC00'
]

def profit_color(v):
    percent = float(v)*100/total
    is_positive = percent > 0
    i = percent if is_positive else -percent
    index = 0
    if i <=0.05:
        index = 5
    elif i <= 1:
        index = 6
    elif i <= 3:
        index = 7
    elif i <= 5:
        index = 8
    else:
        index = 9
    if not is_positive:
        index = 9 - index
    return 'background-color: ' + color_list[index]


def get_underlying_price(exchange, symbol):
    xueqiu_client = XueQiuDataParser()
    ticks = xueqiu_client.history(symbol='SH510300', exchange='SH', period = 'day')
    return ticks.close()[len(ticks.close())-1]

def option_data(exchange, symbol, expiry_date_list, filter_strike):
    underlying_price = get_underlying_price(exchange, symbol)
    return underlying_price, get_options(symbol, underlying_price, expiry_date_list, filter_strike)

greeks_name = ['price', 'iv', 'delta', 'gamma', 'theta', 'vega']
greeks_head = greeks_name[::-1] + ['strike'] + greeks_name
def setup_ui(df):
    out = widgets.Output()
    with out:
        display(df)
    return out

def option_greeks(option):
    greeks = [option.price, option.iv, option.delta, option.gamma, option.theta, option.vega]
    return ['{:0.4f}'.format(g) for g in greeks]

def print_option_chain(chain):
    chain_data = []
    for straddle in chain.straddles:
        c = straddle.call
        p = straddle.put
        chain_data.append(option_greeks(straddle.call)[::-1] + [straddle.strike] + option_greeks(straddle.put))
    return pd.DataFrame(chain_data, columns=greeks_head)

def option_tab(option_map):
    option_chain_outputs = [setup_ui(print_option_chain(chain)) for chain in option_map.chains]
    tab = widgets.Tab()
    tab.children = option_chain_outputs
    for i in range(len(option_map.chains)):
        tab.set_title(i, option_map.chains[i].time_to_expiry.strftime('%Y/%m/%d'))
    return tab

# 计算某个行权价下的期权组合的收益情况
def underlying_profit(x, positions, commision_each_unit):
    total = 0
    commission = 0
    for p in positions:
        if p[0] == 'underlying':
            total += p[1]*(x-p[2])
        else:
            option_type = p[1]
            nums = p[4]
            is_buy = p[4] > 0
            strike = p[3]
            option_price = p[5]
            if option_type == 'c':
                if x <= strike:
                    total += -option_price * nums
                else:
                    total += (x - option_price - strike) * nums
            else:
                if x >= strike:
                    total += -option_price * nums
                else:
                    total += (strike - option_price - x) * nums
            unit = abs(nums)/10000
            unit_vol = unit * 2 if is_buy else unit
            commission += unit_vol * commision_each_unit
        total -= commission
    return total

def update_price(positions, option_map, underlying_price):
    for p in positions:
        if p[0] == 'underlying' and len(p) == 2:
            p.append(underlying_price)
        elif p[0] == 'option' and len(p) == 5:
            o = option_map.get_option(p[2], p[3], p[1])
            p.append(o.price)

def show_options_payoff(positions, option_map, underlying_price):
    update_price(positions, option_map, underlying_price)

    underlying = Underlying(price=underlying_price)
    greeks = calculate_positions(underlying, option_map, positions)
    result = [
        ['变动1'] + ['{:0.2f}'.format(greek) for greek in greeks][0:5],
        ['变动0.0001'] + ['{:0.6f}'.format(greek/10000) for greek in greeks][0:5]
    ]
    profit_column = ['价格变动', 'total_delta', 'delta','gamma', 'theta', 'vega']
    print('profit={:0.2f}, profit_without_underlying={:0.2f}'.format(greeks[5], greeks[6]))    
    display(pd.DataFrame(result, columns=profit_column))
    
    x = pd.Series(np.arange(4.0, 6.0, 0.0001))
    y = x.map(lambda a: underlying_profit(a, positions, 5))
    zero_line = x.map(lambda a: 0)
    fig = go.Figure(data=[go.Scatter(x=x, y=y, name='payoff'), go.Scatter(x=x, y=zero_line, name='zero')])
#     fig = go.Figure(data=[go.line(x=x, y=y, name='payoff'), go.line(x=x, y=zero_line, name='zero')])
    fig.add_vline(x=underlying_price, line_width=1, line_dash="dash", line_color="green")
    fig.show()
    
    iv_diff_list = [-0.2, -0.15, -0.10, -0.07, -0.04, -0.03, -0.02, -0.015, -0.01, -0.005, 0, 0.005, 0.01, 0.015, 0.02, 0.03, 0.04, 0.07, 0.10, 0.15, 0.2]
    underlying_diff_list = [-0.07, -0.06, -0.05, -0.04, -0.03, -0.02, -0.015, -0.01, -0.005, 0, 0.005, 0.01, 0.015, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07]
    matrix = risk_matrix(underlying_price, iv_diff_list, underlying_diff_list, option_map, positions)
    table_head = ['标的变动\\IV'] + iv_diff_list
    option_risk_matrix = pd.DataFrame(matrix, columns=table_head)
    display(option_risk_matrix.style.applymap(profit_color, subset=table_head[1:]))



In [ ]:
symbol = 'SH510300'
exchange = 'SH'
expiry_list = [
    '2021-06-23',
    '2021-07-28',
    '2021-09-22',
    '2021-12-22'
]
expiry_date_list = [datetime.date.fromisoformat(expiry) for expiry in expiry_list]
# option_map = get_options(symbol, underlying_price, expiry_date_list)
underlying_price, option_map = option_data(exchange, symbol, expiry_date_list, True)
print(underlying_price)

option_tab(option_map)

In [ ]:
total = 60000
## 蝶式期权
# positions = [
#     ['option', 'c', expiry_date_list[1], 5.0, 10000],
#     ['option', 'c', expiry_date_list[1], 5.25, -20000],
#     ['option', 'c', expiry_date_list[1], 5.5, 10000],
# ]


positions = [
    ('underlying', 17000, 5.019),
    ('option', 'p', expiry_date_list[1], 4.9, -30000, 0.0500),
    ('option', 'p', expiry_date_list[1], 5.25, -10000, 0.1497),
    ('option', 'c', expiry_date_list[1], 5.25, 10000, 0.0766),
    ('option', 'c', expiry_date_list[1], 5.5, -20000, 0.0205),
    ('option', 'c', expiry_date_list[1], 5.75, 10000, 0.0084),
]
# positions = [
#     ['option', 'c', expiry_date_list[1], 5.25, -10000],
#     ['option', 'c', expiry_date_list[1], 5.5, 20000],
# ]


# positions = [
#     ['option', 'c', expiry_date_list[0], 5.0, -10000],
#     ['option', 'c', expiry_date_list[0], 5.25, 20000],
#     ['option', 'c', expiry_date_list[0], 5.5, -10000],
# ]
show_options_payoff(positions, option_map, underlying_price)

In [ ]:
total = 60000
positions = [
#     ('underlying', 17000, 5.019),
    ('option', 'p', expiry_date_list[1], 4.9, -30000, 0.0500),
    ('option', 'p', expiry_date_list[1], 5.25, -10000, 0.1497),
    ('option', 'c', expiry_date_list[1], 5.25, 10000, 0.0766),
    ('option', 'c', expiry_date_list[1], 5.5, -20000, 0.0205),
    ('option', 'c', expiry_date_list[1], 5.75, 10000, 0.0084),
]
show_options_payoff(positions, option_map, underlying_price)